In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

Using matplotlib backend: Qt5Agg


In [8]:
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' %n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]))
            tf.summary.histogram(layer_name+'/weights',Weights)
        with tf.name_scope('biaes'):
            biaes = tf.Variable(tf.zeros([1, out_size])) + 0.1  #bias一般初始不为0
            tf.summary.histogram(layer_name+'/biaes',biaes)
        with tf.name_scope('Wx_plus_bias'):
            Wx_plus_bias = tf.matmul(inputs, Weights) + biaes
            #tf.histogram_summary(layer_name+'/Wx_plus_bias',Weights)
        if activation_function is None:
            outputs = Wx_plus_bias
        else:
            outputs = activation_function(Wx_plus_bias)
            tf.summary.histogram(layer_name+'/outputs',outputs)
        return outputs

In [9]:
x_data = np.linspace(-1,1,300)[:, np.newaxis]  #300行1列 np.linspace生成-1到1 30 0个数的等差数列
noise = np.random.normal(0, 0.05, x_data.shape)  #格式和x_data一样的格式
y_data = np.square(x_data) - 0.5 + noise

In [10]:
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input') #多少行都行
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)  #输入层1个，隐藏层10个
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),reduction_indices=[1]))
    tf.summary.scalar('loss', loss)
#reduction_indices 表示压缩的方向 

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init = tf.initialize_all_variables()
sess = tf.Session()
merged = tf.summary.merge_all()
#writer = tf.train.write_graph(sess.graph,"logs/",name='graph')
writer = tf.summary.FileWriter("logs/",sess.graph)
sess.run(init)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.scatter(x_data, y_data)
plt.ion()
plt.show()

for i in range(1000):
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i%50 == 0 :
        #print(sess.run(loss, feed_dict={xs:x_data, ys:y_data}))
        result = sess.run(merged, 
                          feed_dict={xs: x_data, ys: y_data})
        
        writer.add_summary(result, i)
        
#         try:
#             ax.lines.remove(lines[0]) 
#         except Exception:
#             pass
        
#         prediction_value = sess.run(prediction, feed_dict={xs:x_data})
#         lines = ax.plot(x_data, prediction_value, 'r-', lw=5)
#         plt.pause(0.1)
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.
